# Optimisation
This notebook presents a procedure to optimize scenarios within AeroMAPS.

## Load

In [ ]:
%matplotlib widget
from aeromaps import create_process
from aeromaps.core.models import models_optim_simple

from aeromaps.core.gemseo import CustomDataConverter

from gemseo.algos.design_space import DesignSpace

# from gemseo.algos.opt.nlopt.settings.nlopt_cobyla_settings import NLOPT_COBYLA_Settings
from gemseo.algos.opt.scipy_local.settings.slsqp import SLSQP_Settings


import gemseo as gm
import warnings

from aeromaps.utils.functions import custom_logger_config

custom_logger_config(gm.configure_logger())

## Models

In [ ]:
models = {
    "models_optim_simple": models_optim_simple,
}

## Process, data and compute

In [ ]:
process = create_process(models=models)

## Constraints definition

These standard constraints are defined in the source code; but as explained in create_a_custom_process, one can define other constraints directly, and add these to the set of models used.

In [ ]:
# Carbon budget share
process.parameters.aviation_carbon_budget_objective = 3.6

# Biomass availability
process.parameters.biomass_availability_constraint_trajectory_reference_years = [
    2020,
    2030,
    2040,
    2050,
]
process.parameters.biomass_availability_constraint_trajectory_reference_years_values = [
    62.8,
    72.1,
    97.8,
    101.8,
]

# Electricity availability
process.parameters.electricity_availability_constraint_trajectory_reference_years = [
    2020,
    2030,
    2040,
    2050,
]
process.parameters.electricity_availability_constraint_trajectory_reference_years_values = [
    102.0456,
    137.5452,
    212.7996,
    276.6168,
]

# Volume and rate ramp up constraints
process.parameters.volume_ramp_up_constraint_biofuel = 0.3
process.parameters.rate_ramp_up_constraint_biofuel = 0.2

process.parameters.volume_ramp_up_constraint_electrofuel = 0.3
process.parameters.rate_ramp_up_constraint_electrofuel = 0.2


# Other settings

## Carbon budgets and Carbon Dioxide Removal [GtCO2]
process.parameters.net_carbon_budget = 850.0
process.parameters.carbon_dioxyde_removal_2100 = 285.0


## Aviation share of the global (equivalent) carbon budget [%]
process.parameters.aviation_carbon_budget_allocated_share = 3.6
process.parameters.aviation_equivalentcarbonbudget_allocated_share = 5.1

## Aviation share of the global energy resources (biomass and electricity) [%]
process.parameters.aviation_biomass_allocated_share = 15.0
process.parameters.aviation_electricity_allocated_share = 8.0


### Biofuel and efuel reference years associated with the optim variables

process.parameters.biofuel_share_reference_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]
process.parameters.electrofuel_share_reference_years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]

### Optimisation problem setup with GEMSEO

In [ ]:
process.setup()

# Create a GEMSEO scenario

process.gemseo_settings["scenario_type"] = "MDO"
process.gemseo_settings["formulation"] = "MDF"


# define the design sapce: electrofuel and biofuel blending mandates

# NB: possible to modify DropinFuelDistribution to directly fix the first two values instead of creating frozen varaibles here, to reduce the design space size.

design_space = DesignSpace()
design_space.add_variable(
    "electrofuel_share_reference_years_values",
    size=7,
    lower_bound=[0, 0, 1e-17, 1e-17, 1e-17, 1e-17, 1e-17],
    upper_bound=[0.01, 0.01, 100, 100, 100, 100, 100],
    value=[0.001, 0.001, 9.13249637, 14.68171346, 24.29143904, 41.14065492, 41.87052262],
)


design_space.add_variable(
    "biofuel_share_reference_years_values",
    size=7,
    lower_bound=[0, 2, 1e-17, 1e-17, 1e-17, 1e-17, 1e-17],
    upper_bound=[0.01, 2.01, 100, 100, 100, 100, 100],
    value=[0.001, 2.001, 10.85689249, 16.98435693, 29.28238527, 32.74786858, 29.99980536],
)


process.gemseo_settings["design_space"] = design_space


# Define the objective
objective_name = "cumulative_total_airline_cost_discounted_obj"
process.gemseo_settings["objective_name"] = objective_name


process.create_gemseo_scenario()
# Make optimisation objective values in the 1-10 interval
process.scenario.formulation.optimization_problem.objective = (
    process.scenario.formulation.optimization_problem.objective * 1e-13
)


# Add constraints
all_constraints = [
    "aviation_carbon_budget_constraint",
    "blend_completeness_constraint",
    "electricity_trajectory_constraint",
    "biomass_trajectory_constraint",
    # "electrofuel_use_growth_constraint",
    # "biofuel_use_growth_constraint",
    # "biofuel_use_no_degrowth_constraint",
    # "electrofuel_use_no_degrowth_constraint",
]


for constraint in all_constraints:
    process.scenario.add_constraint(constraint, constraint_type="ineq")


process.scenario.set_differentiation_method("finite_differences")

# Optim algo: COBYLA, a gradient-free non linear solver ==> Require installation of NLOP; better than SLSQP to start with
# cobyla_settings = NLOPT_COBYLA_Settings(
#     max_iter=3,  # CAUTION, 3 set for quick automated test , use something like 200,
#     ftol_rel=0.001,
#     ftol_abs=0.001,
#     ineq_tolerance=0.015,
#     normalize_design_space=True,
#     init_step=0.1,
# )


slsqp_settings = SLSQP_Settings(
    max_iter=2,  # CAUTION, 2 set for quick automated test , use something like 200,
    ftol_rel=0.001,
    ftol_abs=0.001,
    ineq_tolerance=0.015,
    normalize_design_space=True,
)


process.gemseo_settings["algorithm"] = slsqp_settings  # cobyla_settings


# Adding design variables to the set of list types varaible (they are declared as ndarray but needed as lists within aeromaps functions)

CustomDataConverter._list_names.update(process.scenario.get_optim_variable_names())

In [ ]:
warnings.filterwarnings("ignore")
process.compute()

## Results

In [ ]:
process = process

In [ ]:
process.scenario.post_process(
    post_name="OptHistoryView",
    save=False,
    show=True,
)

In [ ]:
process.scenario.get_result().design_variable_names_to_values

In [ ]:
process.plot("dropin_fuel_shares")